In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn as nn
import os
from transformers import pipeline, AutoTokenizer

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
CHECKPOINT_FOLDER = 'deepHW4/Q3/checkpoints'
checkpoint_folder_path = os.path.join('/content/drive/My Drive', CHECKPOINT_FOLDER)
os.makedirs(checkpoint_folder_path, exist_ok=True)

In [5]:
dataset = 'ferdousi.txt'
input_texts = []
output_texts = []
lines = None
with open(dataset, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
lines = lines[2:]
for i in range(0, len(lines)-3, 2):
    input_text = lines[i] + '  ' + lines[i+1]
    output_text = lines[i+2] + '  ' + lines[i+3]
    # input_text = lines[i]
    # output_text = lines[i+1]
    input_texts.append(input_text)
    output_texts.append(output_text)

idx = 2
print(f'{input_texts[idx]}')
print(f'{output_texts[idx]}')
# print(f'{input_texts[idx]} {output_texts[idx]} ')

خداوند کیوان و گردان سپهر  فروزنده ماه و ناهید و مهر
ز نام و نشان و گمان برترست  نگارندهٔ بر شده پیکرست


In [6]:
# Separate into training and test sets
input_train, input_test, output_train, output_test = train_test_split(input_texts, output_texts, test_size=0.2, random_state=42)
idx_train = 0
idx_test = 0
print(f'Training Input: {input_train[idx_train]}')
print(f'Training Output: {output_train[idx_train]}')
print(f'Test Input: {input_test[idx_test]}')
print(f'Test Output: {output_test[idx_test]}')

Training Input: دبیر جهاندیده را پیش خواند  ازان چاره و چنگ چندی براند
Training Output: بر تخت بنشست فرخ دبیر  قلم خواست و قرطاس و مشک و عبیر
Test Input: گنهکارتر چیز مردم بود  که از کین و آزش خرد گم بود
Test Output: چو خواهی که این را بدانی درست  تن خویشتن را نگه کن نخست


In [7]:
class Custom_dataset(Dataset):
    def __init__(self, input, output, tokenizer, max_length=20):
        self.input = input
        self.output = output
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        input = self.input[idx]
        output = self.output[idx]

        input_tokens = self.tokenizer.encode(input, max_length=self.max_length, truncation=True, padding='max_length')
        output_tokens = self.tokenizer.encode(output, max_length=self.max_length, truncation=True, padding='max_length')
        in_tok = torch.tensor(input_tokens, dtype=torch.long)
        ou_tok = torch.tensor(output_tokens, dtype=torch.long)
        return in_tok, ou_tok


In [8]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/gpt2-fa")
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/875k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.75M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/14.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/104 [00:00<?, ?B/s]

In [9]:
train_dataset = Custom_dataset(input_train, output_train, tokenizer)
test_dataset = Custom_dataset(input_test, output_test, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [10]:
for batch in train_dataloader:
    input_batch, output_batch = batch
    print(input_batch.shape)
    print(output_batch.shape)
    break
for batch in test_dataloader:
    x, y = batch
    print(x.shape)
    print(y.shape)
    break

torch.Size([64, 20])
torch.Size([64, 20])
torch.Size([64, 20])
torch.Size([64, 20])


In [11]:
x[63]

tensor([ 3718,   521,  3490,  4828,   298,  1088,   231, 33475,  2203,  5146,
          325, 18897,     5,     5,     5,     5,     5,     5,     5,     5])

In [12]:
# config = AutoConfig.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, config=config)
# # type(model)
generator = pipeline(task= "text-generation", model= "HooshvareLab/gpt2-fa")

pytorch_model.bin:   0%|          | 0.00/485M [00:00<?, ?B/s]

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(generator.model.parameters(), lr=1e-3)

In [13]:
# generator.model.to(device)


# # Training loop
# generator.model.train()
# epochs = 15
# for epoch in range(epochs):
#     total_loss = 0
#     for batch in train_dataloader:
#         x, y = batch
#         x = x.to(device)
#         y = y.to(device)

#         x1 = x.reshape((len(x),-1))
#         y = y.view(-1)
#         s = generator.model.transformer(x1).last_hidden_state
#         s_1 = generator.model.lm_head(s)
#         logits = s_1.view(-1, s_1.shape[-1])
#         loss = criterion(logits, y)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     average_loss = total_loss / len(train_dataloader)
#     print(f'Epoch {epoch + 1}/{epochs}, Average Loss: {average_loss}')
#     # checkpoint_path = os.path.join(checkpoint_folder_path, f"model_checkpoint_epoch_{epoch}.pt")
#     # torch.save({generator}, checkpoint_path)

Epoch 1/15, Average Loss: 4.967110163923623
Epoch 2/15, Average Loss: 4.808520994324615
Epoch 3/15, Average Loss: 4.768484713758632
Epoch 4/15, Average Loss: 4.731959489616603
Epoch 5/15, Average Loss: 4.692550623090563
Epoch 6/15, Average Loss: 4.641783749615705
Epoch 7/15, Average Loss: 4.576257443082506
Epoch 8/15, Average Loss: 4.496739659332423
Epoch 9/15, Average Loss: 4.393950109896452
Epoch 10/15, Average Loss: 4.266253899452959
Epoch 11/15, Average Loss: 4.104981686949922
Epoch 12/15, Average Loss: 3.9066189901840285
Epoch 13/15, Average Loss: 3.670440100625325
Epoch 14/15, Average Loss: 3.408598049827244
Epoch 15/15, Average Loss: 3.1219570832551966


In [19]:
checkpoint_path = os.path.join(checkpoint_folder_path, "model_checkpoint3.pt")
checkpoint = torch.load(checkpoint_path)
generator.model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [20]:
# Training loop
generator.model.to(device)
generator.model.train()
epochs = 1
for epoch in range(epochs):
    total_loss = 0
    for batch in train_dataloader:
        x, y = batch
        x = x.to(device)
        y = y.to(device)

        x1 = x.reshape((len(x),-1))
        x1 = x1.to(device)
        y = y.view(-1)
        s = generator.model.transformer(x1).last_hidden_state
        s_1 = generator.model.lm_head(s)
        logits = s_1.view(-1, s_1.shape[-1])
        loss = criterion(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs}, Average Loss: {average_loss}')

Epoch 1/1, Average Loss: 0.7706407247150194


In [30]:
# checkpoint_path = os.path.join(checkpoint_folder_path, f"model_checkpoint1.pt")
# torch.save(generator.model.state_dict(),'model_checkpoint3.pth')
checkpoint_path = os.path.join(checkpoint_folder_path, f"model_checkpoint3.pt")
torch.save({
    'model_state_dict': generator.model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, checkpoint_path)

به علت حجم بالای بهترین مدل بنده ، نتوانستم آن را در فایل تحولی آپلود کنم و فقط کد نوشته شده را آپلود کردم.

In [15]:
generator.model.eval()
generator.model.to(device)
input_text = "به نام خداوند نام و خداوند جای"
input_tokens = tokenizer.encode(input_text, return_tensors="pt", max_length=20, truncation=True, padding='max_length')
input_tokens = input_tokens.to(device)
with torch.no_grad():
    scores = generator.model(input_tokens).logits
outs = scores.argmax(2)
output_text = tokenizer.decode(outs[0], skip_special_tokens=True)
print("Generated Text:")
print(output_text)

Generated Text:
چو پستان گا بر خداوند دست    ن پیش پیش به بود


In [16]:
x, y = next(iter(test_dataloader))
x = x.to(device)
y = y.to(device)
scores = generator.model(x[:10]).logits
outs = scores.argmax(2)
for i in range(10):
  input = tokenizer.decode(x[i])
  input = ' '.join(input.split('<|endoftext|>'))
  output = tokenizer.decode(outs[i])
  output = ' '.join(output.split('<|endoftext|>'))
  print(input)
  print(output)
  print()

گنهکارتر چیز مردم بود  که از کین و آزش خرد گم بود   
ز و که از سود کاین   از آرزوها نیستش هنگام درست و     

گرفته سپر پیش و ژوپین بدست  ببالا نهاده سر از جای پست    
ز گونه گر آن بردل   زستند گریزان و فرو       

بزد مهره در جام و برخاست غو  برآمد ز هر دو سپه دار و رو   
چوپه خروش کوس با کر   همان بر و نامور و       

سپه کش پشوتن به قلب اندرون  سپاهی همه دست شسته به خون    
چوشکر به در گفت به هزار   با ستاره چو چو گل کوس     

ز گیتی بیامد ترا برگزید  چنان کز ره نامداران سزید      
چو از به و اسفندیار اند  هندوستان شد همان   خواه      

ببخشد بپرهیزد از مهر گنج  نبندد دل اندر سرای سپنج   
چو یزدانٔ جامهارد به   ز از رانده به       

که چندین بزرگی و گنج و سپاه  گرانمایه اسپان و تخت و کلاه     
چو چیره پاسخ گفتار گونه چو   که و تاج داری چو       

نثارش کن از گوهر شاهوار  سه یاقوت سرخ از در شهریار      
زامید گرقب که شویوار   سهٔ بر از بر بر     

به ایرج نگه کرد یکسر سپاه  که او بد سزاوار تخت و کلاه     
چو چهره خواهد شاه به به تاج  که که از آن سر بختد بخت    

بیک هفته بودش هم آنجا 

In [17]:
import math

# Testing loop
generator.model.to(device)
generator.model.eval()
total_loss = 0
total_tokens = 0
num_samples = 0

with torch.no_grad():
    for batch in test_dataloader:
        x, y = batch
        x, y = x.reshape((len(x), -1)).to(device=device), y.to(device)

        s = generator.model.transformer(x).last_hidden_state
        s_1 = generator.model.lm_head(s)
        logits = s_1.view(-1, s_1.shape[-1])

        loss = criterion(logits, y.view(-1))

        total_loss += loss.item()
        num_samples += 1

# Calculate perplexity
average_loss = total_loss / num_samples
perplexity = math.exp(average_loss)

print(f'Perplexity on the test set: {perplexity}')

Perplexity on the test set: 35116.11208978624
